In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%pwd

'/content'

In [0]:
%cd drive

/content/drive


In [0]:
%cd My\ Drive

/content/drive/My Drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.read_csv("cgfinal414.csv")

In [0]:
df_cgd=df.iloc[:,:-5]

In [0]:
df_cgd=df_cgd.dropna()

In [0]:
df_cgd=df_cgd.drop(['id'],axis=1)

In [0]:

one_hot = pd.get_dummies(df_cgd['StudyArm'])
# Drop column B as it is now encoded
df_cgd = df_cgd.drop('StudyArm',axis = 1)
# Join the encoded df
df_cgd = df_cgd.join(one_hot)

In [0]:
dtrain=df_cgd

features_selected = [ 'Control','GA', 'Age', 'Gender', 'gradecat', 'incomecat',
       'racecat', 'Living', 'relationship_cat', 'GI_Lung', 'TTC3', 'TTC5',
       'TTC6a', 'TTC6b', 'TTC6c', 'TTC6d', 'distress',
       'ImpairedPolypharmacy', 'cognition', 'nutrition',
       'phy_performance', 'function', 'ImpairedCom', 'psychological',
       'ImpairedMS', 'calcimpairedCom', 'cgdistress']
#for now we use all the features for regression
#features_selected=[features_selected[i] for i in [38,42,45,47,5,19,13,22,25,20]]

Xtrain = []
for i in features_selected:
    Xtrain.append(dtrain[i].values)

Xtrain = np.array(Xtrain)
Xtrain=Xtrain.T
predicted_feature = ['cgdistress']
Ytrain= []
for i in predicted_feature:
    Ytrain.append(dtrain[i].values)


Ytrain = np.array(Ytrain)
Ytrain=Ytrain.T.ravel()

# Xtest = []
# for i in features_selected:
#     Xtest.append(dtest[i].values)

# Xtest = np.array(Xtest)
# Xtest=Xtest.T
# predicted_feature = ['def_total']
# Ytest= []
# for i in predicted_feature:
#     Ytest.append(dtest[i].values)


# Ytest = np.array(Ytest)
# Ytest=Ytest.T.ravel()

In [0]:
Ytrain.shape

(398,)

In [0]:
from sklearn.ensemble import RandomForestClassifier as RF

from sklearn.model_selection import GridSearchCV
model = RF(n_jobs=-1, random_state=42, verbose=2)

grid = {
        'max_features': [0.07,0.15, 0.25],
        'n_estimators': [10,25,45, 60,110, 148, 200],
        'min_samples_split': [2, 5, 8, 12]
       }

rf_gridsearch = GridSearchCV(estimator=model, param_grid=grid, n_jobs=5, 
                             cv=5, verbose=5, return_train_score=True,scoring="accuracy")

rf_gridsearch.fit(Xtrain,Ytrain)

df_gridsearch = pd.DataFrame(rf_gridsearch.cv_results_)

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    2.8s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:   11.2s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   24.1s
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed:   45.2s


building tree 1 of 45building tree 4 of 45

building tree 2 of 45
building tree 3 of 45
building tree 5 of 45
building tree 6 of 45
building tree 7 of 45
building tree 8 of 45
building tree 9 of 45
building tree 10 of 45
building tree 11 of 45
building tree 12 of 45
building tree 13 of 45
building tree 14 of 45
building tree 15 of 45
building tree 16 of 45building tree 17 of 45

building tree 18 of 45
building tree 19 of 45
building tree 20 of 45building tree 21 of 45

building tree 22 of 45
building tree 23 of 45
building tree 24 of 45
building tree 25 of 45
building tree 26 of 45
building tree 27 of 45building tree 28 of 45

building tree 29 of 45building tree 30 of 45

building tree 31 of 45
building tree 32 of 45
building tree 33 of 45
building tree 34 of 45
building tree 35 of 45
building tree 36 of 45
building tree 37 of 45
building tree 38 of 45
building tree 39 of 45
building tree 40 of 45
building tree 41 of 45
building tree 42 of 45
building tree 43 of 45
building tree 44 of 

[Parallel(n_jobs=5)]: Done 420 out of 420 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    0.1s finished


In [0]:
df_gridsearch

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.131444,0.018653,0.105998,0.004261,0.07,2,10,"{'max_features': 0.07, 'min_samples_split': 2,...",0.9250,0.8250,0.850,0.835443,0.873418,0.861772,0.035545,84,1.0,0.996855,0.993711,0.99373,0.996865,0.996232,0.00235
1,0.123101,0.006395,0.109664,0.002818,0.07,2,25,"{'max_features': 0.07, 'min_samples_split': 2,...",0.9625,0.9250,0.975,0.962025,0.962025,0.957310,0.016901,73,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
2,0.209546,0.042899,0.117708,0.007732,0.07,2,45,"{'max_features': 0.07, 'min_samples_split': 2,...",0.9875,0.9625,0.975,0.949367,0.962025,0.967278,0.012960,68,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
3,0.303973,0.046878,0.117751,0.007475,0.07,2,60,"{'max_features': 0.07, 'min_samples_split': 2,...",0.9625,0.9500,0.975,0.936709,0.949367,0.954715,0.013016,77,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
4,0.605665,0.109159,0.118481,0.008485,0.07,2,110,"{'max_features': 0.07, 'min_samples_split': 2,...",0.9750,0.9750,0.975,0.949367,0.974684,0.969810,0.010222,63,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.190946,0.065034,0.112388,0.009685,0.25,12,45,"{'max_features': 0.25, 'min_samples_split': 12...",1.0000,1.0000,1.000,1.000000,1.000000,1.000000,0.000000,1,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
80,0.340690,0.108746,0.107188,0.003519,0.25,12,60,"{'max_features': 0.25, 'min_samples_split': 12...",1.0000,1.0000,1.000,1.000000,1.000000,1.000000,0.000000,1,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
81,0.602014,0.073838,0.124452,0.009530,0.25,12,110,"{'max_features': 0.25, 'min_samples_split': 12...",1.0000,1.0000,1.000,1.000000,1.000000,1.000000,0.000000,1,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000
82,1.084371,0.241009,0.173946,0.054228,0.25,12,148,"{'max_features': 0.25, 'min_samples_split': 12...",1.0000,1.0000,1.000,1.000000,1.000000,1.000000,0.000000,1,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,0.00000


In [0]:
from sklearn.ensemble import RandomForestClassifier as RF

from sklearn.model_selection import GridSearchCV
model = RF(n_jobs=-1, random_state=42, verbose=2)

grid = {
        'max_features': [0.07,0.13, 0.15, 0.25,'auto','sqrt'],
        'n_estimators': [10,25,45, 60,110, 148, 200],
        'min_samples_split': [2, 3, 5, 8, 12],
         'min_samples_leaf': [1, 2, 4,8],
        'bootstrap': [True, False],
        'max_depth': [10, 20,40, 60]
       }

rf_gridsearch = GridSearchCV(estimator=model, param_grid=grid, n_jobs=5, 
                             cv=5, verbose=5, return_train_score=True,scoring="accuracy")

rf_gridsearch.fit(Xtrain,Ytrain)

df_gridsearch1 = pd.DataFrame(rf_gridsearch.cv_results_)

Fitting 5 folds for each of 6720 candidates, totalling 33600 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:    9.7s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   22.8s
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed:   43.1s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  1.1min
[Parallel(n_jobs=5)]: Done 638 tasks      | elapsed:  1.6min
[Parallel(n_jobs=5)]: Done 872 tasks      | elapsed:  2.2min
[Parallel(n_jobs=5)]: Done 1142 tasks      | elapsed:  2.9min
[Parallel(n_jobs=5)]: Done 1448 tasks      | elapsed:  3.7min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed:  4.6min
[Parallel(n_jobs=5)]: Done 2168 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done 2582 tasks      | elapsed:  6.6min
[Parallel(n_jobs=5)]: Done 3032 tasks      | elapsed:  7.7min
[Parallel(n_jobs=5)]: Done 3518 tasks      | elapsed:  9.0min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed: 10.3min
[P

building tree 1 of 45building tree 2 of 45

building tree 3 of 45
building tree 4 of 45
building tree 5 of 45
building tree 6 of 45
building tree 7 of 45
building tree 8 of 45
building tree 9 of 45
building tree 10 of 45
building tree 11 of 45
building tree 12 of 45building tree 13 of 45

building tree 14 of 45
building tree 15 of 45
building tree 16 of 45
building tree 17 of 45
building tree 18 of 45
building tree 19 of 45
building tree 20 of 45
building tree 21 of 45
building tree 22 of 45
building tree 23 of 45
building tree 24 of 45
building tree 25 of 45
building tree 26 of 45building tree 27 of 45

building tree 28 of 45building tree 29 of 45

building tree 30 of 45
building tree 31 of 45
building tree 32 of 45
building tree 33 of 45
building tree 34 of 45
building tree 35 of 45
building tree 36 of 45
building tree 37 of 45
building tree 38 of 45
building tree 39 of 45
building tree 40 of 45
building tree 41 of 45
building tree 42 of 45
building tree 43 of 45
building tree 44 of 

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    0.1s finished


In [0]:
from sklearn.ensemble import RandomForestClassifier as RF

from sklearn.model_selection import GridSearchCV
model = RF(n_jobs=-1, random_state=42, verbose=2)

grid = {
        'max_features': [0.07,0.13, 0.15, 0.25,'auto','sqrt'],
        'n_estimators': [10,25,45, 60,110, 148, 200],
        'min_samples_split': [2, 3, 5, 8, 12],
         'min_samples_leaf': [1, 2, 4,8],
        'bootstrap': [True, False],
        'max_depth': [10, 20,40, 60]
       }

rf_gridsearch = GridSearchCV(estimator=model, param_grid=grid, n_jobs=5, 
                             cv=5, verbose=5, return_train_score=True,scoring="accuracy")

rf_gridsearch.fit(Xtrain,Ytrain)

df_gridsearch1 = pd.DataFrame(rf_gridsearch.cv_results_)

Fitting 5 folds for each of 6720 candidates, totalling 33600 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:    9.7s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   22.8s
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed:   43.1s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  1.1min
[Parallel(n_jobs=5)]: Done 638 tasks      | elapsed:  1.6min
[Parallel(n_jobs=5)]: Done 872 tasks      | elapsed:  2.2min
[Parallel(n_jobs=5)]: Done 1142 tasks      | elapsed:  2.9min
[Parallel(n_jobs=5)]: Done 1448 tasks      | elapsed:  3.7min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed:  4.6min
[Parallel(n_jobs=5)]: Done 2168 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done 2582 tasks      | elapsed:  6.6min
[Parallel(n_jobs=5)]: Done 3032 tasks      | elapsed:  7.7min
[Parallel(n_jobs=5)]: Done 3518 tasks      | elapsed:  9.0min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed: 10.3min
[P

building tree 1 of 45building tree 2 of 45

building tree 3 of 45
building tree 4 of 45
building tree 5 of 45
building tree 6 of 45
building tree 7 of 45
building tree 8 of 45
building tree 9 of 45
building tree 10 of 45
building tree 11 of 45
building tree 12 of 45building tree 13 of 45

building tree 14 of 45
building tree 15 of 45
building tree 16 of 45
building tree 17 of 45
building tree 18 of 45
building tree 19 of 45
building tree 20 of 45
building tree 21 of 45
building tree 22 of 45
building tree 23 of 45
building tree 24 of 45
building tree 25 of 45
building tree 26 of 45building tree 27 of 45

building tree 28 of 45building tree 29 of 45

building tree 30 of 45
building tree 31 of 45
building tree 32 of 45
building tree 33 of 45
building tree 34 of 45
building tree 35 of 45
building tree 36 of 45
building tree 37 of 45
building tree 38 of 45
building tree 39 of 45
building tree 40 of 45
building tree 41 of 45
building tree 42 of 45
building tree 43 of 45
building tree 44 of 

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    0.1s finished


In [0]:
df_gridsearch1.to_csv("result_rf1.csv")

In [0]:
from sklearn.svm import SVC 

from sklearn.model_selection import GridSearchCV
modelsvm = SVC(random_state=42, verbose=2)



param_gridsvm = {'C': [0.1, 1, 10,20,50, 100,200,500,1000],  
              'gamma': [1,0.5, 0.1, 0.01, 0.05, 0.001,0.005, 0.0001,0.0005], 
              'kernel': ['rbf']}  
  


svmgridsearch = GridSearchCV(estimator=modelsvm, param_grid=param_gridsvm, n_jobs=-1, 
                             cv=5, verbose=2, return_train_score=True,scoring="accuracy")

svmgridsearch.fit(Xtrain,Ytrain)

df_gridsearchsvm = pd.DataFrame(svmgridsearch.cv_results_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    3.1s


[LibSVM]

[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:    7.1s finished


In [0]:
df_gridsearchsvm.to_csv("result_svm.csv")

In [0]:
%ls

'0perati0n:Reiten .gdoc'
'2019 CS Coding Challenge Flyer.pdf'
'242 exam4'/
'3.2.20 Final Data Sharing agreemnt Data  Science (1).pdf'
'3.2.20 Final Data Sharing agreemnt Data  Science.docx'
 ACC221.pdf
 a.gdoc
 Analysis.ipynb
 analytics_colloquium_april242019_flier.pdf
 Android/
 app.zip
'Bi-weekly Progress Report (2).pptx'
'Book report.gdoc'
 centered_data.csv
 cfa.zip
 c.gdoc
 cgfinal414.csv
'Chenwei Ma.rar'
 chenweiprojectwork/
 Chenwei_Project.zip
'Chenwei Wu_ Resume.pdf'
 ChenweiWuResume.pdf
'Class notes.gdoc'
 cnn_models/
'code (1).ipynb'
 code.ipynb
'Colab Notebooks'/
 comb
 companylist1.csv
'Copy of slides March2.pptx'
 Cpt/
'curr (1).zip'
 curr.zip
 data_orig.csv
'data_withcolumnsadded_nootheryearnan (1).csv'
 data_withcolumnsadded_nootheryearnan.csv
 demo.mp4
 deployed_models/
 df_gridsearch.csv
 disc
 DSC265HW4.gdoc
 ECON207.pdf
'Econ paper.gdoc'
 edit4.csv
 erika_coder/
'Event Description Form - Projects 2018-2019.gdoc'
'Fall 2017'/
 files/
 FIN205.7z
'finale paperwcw.docx'

In [0]:
####################EXTRAAA##################smote#######

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Xtrain,Ytrain,random_state=1)

In [21]:
X_train.shape

(298, 27)

In [22]:
!pip install imblearn

In [23]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
random_state = 42
modelsvm = Pipeline([
        # Original SMOTE class
        ('smote', SMOTE(random_state=random_state)),
        ('svm', SVC(random_state=42, verbose=2,kernel='rbf'))
    ])

# modelsvm = SVC(random_state=42, verbose=2,kernel='rbf')

from sklearn.model_selection import GridSearchCV



param_gridsvm = {'svm__C': [0.1, 1, 10,20,50, 100,200,500,1000],  
              'svm__gamma': [1,0.5, 0.1, 0.01, 0.05, 0.001,0.005, 0.0001,0.0005]}  
  


svmgridsearch = GridSearchCV(estimator=modelsvm, param_grid=param_gridsvm, n_jobs=-1, 
                             cv=5, verbose=2, return_train_score=True,scoring="accuracy")

svmgridsearch.fit(Xtrain,Ytrain)

df_gridsearchsvmk = pd.DataFrame(svmgridsearch.cv_results_)

In [0]:
df_gridsearchsvmk

In [0]:
listresults=[]

In [0]:

def evaluate_model(model, model_name, params):
    
    grid_search = GridSearchCV(model,params,cv=5,n_jobs=-1, verbose=5, return_train_score=True,scoring="accuracy")
    grid_search.fit(X_train,y_train)
    
    
    test_score=grid_search.score(X_test,y_test)
    listresults.append(pd.DataFrame(grid_search.cv_results_))
    print('Classifier')
    print(model_name)
    print('Test Score')
    print(test_score)
    print("CV Score")
    print(grid_search.best_score_)
    print("best params")
    print(grid_search.best_params_)

In [0]:


from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    
# The function `init_classifiers` returns a list of classifiers to be trained on the datasets
def init_classifiers():
    return([(SVC(), model_names[0], param_grid_svc), 
            (RandomForestClassifier(), model_names[1], param_grid_rf),
           ])

# 'model_names' contains the names  that we will use for the above classifiers
model_names = ['SVM','RF']

# the training parameters of each model
param_grid_svc = [{'C': [0.1, 1, 10,20,50, 100,200,500,1000],  
              'gamma': [1,0.5, 0.1, 0.01, 0.05, 0.001,0.005, 0.0001,0.0005], 
              'kernel': ['rbf'],'random_state':[42]}]
param_grid_rf = [{
        'max_features': [0.07, 0.15, 0.25],
        'n_estimators': [10,45, 60,110, 148],
        'min_samples_split': [2, 3, 5, 8],
         'min_samples_leaf': [1, 2, 4,8],
        'bootstrap': [True, False],
        'max_depth': [10, 20,40],
        'random_state':[42],
        'n_jobs':[-1],
        'verbose':[2]
       }]

In [0]:
for m in init_classifiers():
    evaluate_model(m[0], m[1], m[2])

In [0]:
# Fitting 5 folds for each of 81 candidates, totalling 405 fits
# [Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
# [Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.9s
# [Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:    4.1s finished
# [Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
# Classifier
# SVM
# Test Score
# 0.99
# CV Score
# 1.0
# best params
# {'C': 20, 'gamma': 0.001, 'kernel': 'rbf', 'random_state': 42}
# Fitting 5 folds for each of 1440 candidates, totalling 7200 fits
# [Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.5s
# [Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   14.3s
# [Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   33.3s
# [Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:   59.8s
# [Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.6min
# [Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  2.3min
# [Parallel(n_jobs=-1)]: Done 878 tasks      | elapsed:  3.1min
# [Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed:  4.1min
# [Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed:  5.1min
# [Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  6.3min
# [Parallel(n_jobs=-1)]: Done 2174 tasks      | elapsed:  7.7min
# [Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed:  9.2min
# [Parallel(n_jobs=-1)]: Done 3038 tasks      | elapsed: 10.8min
# [Parallel(n_jobs=-1)]: Done 3524 tasks      | elapsed: 12.5min
# [Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 14.2min
# [Parallel(n_jobs=-1)]: Done 4604 tasks      | elapsed: 16.0min
# [Parallel(n_jobs=-1)]: Done 5198 tasks      | elapsed: 17.9min
# [Parallel(n_jobs=-1)]: Done 5828 tasks      | elapsed: 20.0min
# [Parallel(n_jobs=-1)]: Done 6494 tasks      | elapsed: 22.1min
# [Parallel(n_jobs=-1)]: Done 7196 tasks      | elapsed: 24.4min
# [Parallel(n_jobs=-1)]: Done 7200 out of 7200 | elapsed: 24.4min finished
# [Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
# [Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    0.1s
# [Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    0.1s finished
# [Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
# [Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
# [Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished
# building tree 1 of 45
# building tree 2 of 45
# building tree 3 of 45
# building tree 4 of 45building tree 5 of 45

# building tree 6 of 45
# building tree 7 of 45
# building tree 8 of 45
# building tree 9 of 45
# building tree 10 of 45
# building tree 11 of 45building tree 12 of 45

# building tree 13 of 45
# building tree 14 of 45
# building tree 15 of 45
# building tree 16 of 45
# building tree 17 of 45
# building tree 18 of 45
# building tree 19 of 45
# building tree 20 of 45
# building tree 21 of 45
# building tree 22 of 45
# building tree 23 of 45
# building tree 24 of 45
# building tree 25 of 45
# building tree 26 of 45
# building tree 27 of 45
# building tree 28 of 45
# building tree 29 of 45
# building tree 30 of 45
# building tree 31 of 45
# building tree 32 of 45
# building tree 33 of 45
# building tree 34 of 45building tree 35 of 45

# building tree 36 of 45
# building tree 37 of 45
# building tree 38 of 45
# building tree 39 of 45building tree 40 of 45

# building tree 41 of 45
# building tree 42 of 45
# building tree 43 of 45
# building tree 44 of 45
# building tree 45 of 45
# Classifier
# RF
# Test Score
# 1.0
# CV Score
# 1.0
# best params
# {'bootstrap': True, 'max_depth': 10, 'max_features': 0.15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 45, 'n_jobs': -1, 'random_state': 42, 'verbose': 2}

In [0]:
listresults[0].to_csv("gsearchres1.csv")

In [0]:
listresults[1].to_csv("gsearchres2.csv")